In [1]:
%autosave 0

Autosave disabled


In [2]:
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline
import tensorflow as tf
from tensorflow import keras
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.xception import preprocess_input
from tensorflow.keras.applications.xception import Xception


2021-11-19 11:56:45.615688: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-11-19 11:56:45.615711: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


# Setting up the CNN model

In [3]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from tensorflow.keras.optimizers import SGD



# define cnn model
def define_model():
    model = Sequential()
    model.add(Conv2D(filters = 32, kernel_size = (3,3), 
                 activation ='relu', input_shape = (150,150,3)))
                 
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Flatten())
    model.add(Dense(64, activation = "relu"))
    model.add(Dense(1, activation='sigmoid')) #since is binary classification
    optimizer = SGD(lr=0.002, momentum=0.8)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [4]:
model = define_model()

# Looking for the number of total parameters
print(model.summary())


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 74, 74, 32)       0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 175232)            0         
                                                                 
 dense (Dense)               (None, 64)                11214912  
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 11,215,873
Trainable params: 11,215,873
Non-trainable params: 0
____________________________________________

2021-11-19 11:56:50.886816: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-11-19 11:56:50.886842: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2021-11-19 11:56:50.886866: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (9h8lvp2.usr.local): /proc/driver/nvidia/version does not exist
2021-11-19 11:56:50.887102: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
/media/cap/7fed51bd-a88e-4971-9656-d617655b6312/data/sources/conda/miniconda3/envs/py39/lib/python3.9/sit

In [5]:
# The answer to Q1 is 11,215,873

# Now preparing the data

## For Q2 we are told to do the following

Generators and Training
For the next two questions, use the following data generator for both train and validation:

ImageDataGenerator(rescale=1./255)
We don't need to do any additional pre-processing for the images.
When reading the data from train/val directories, check the class_mode parameter. Which value should it be for a binary classification problem?
Use batch_size=20
For training use .fit() with the following params:

model.fit(
    train_generator,
    steps_per_epoch=100,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=50
)

In [6]:
DATAPATH="/media/cap/7fed51bd-a88e-4971-9656-d617655b6312/data/ML_Data/BootCamp/"

train_gen = ImageDataGenerator(rescale=1./255)
#not sure I need to do it twice
val_gen = ImageDataGenerator(rescale=1./255)

train_ds = train_gen.flow_from_directory(os.path.join(DATAPATH,"train"),
class_mode='binary', batch_size=20, target_size=(150, 150))

validation_ds = val_gen.flow_from_directory(os.path.join(DATAPATH,"validation"),
 class_mode='binary', batch_size=20, target_size=(150, 150))

Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.


In [7]:

# train_gen = ImageDataGenerator(preprocessing_function=preprocess_input)

# train_ds = train_gen.flow_from_directory(
#     os.path.join(DATAPATH,"train"),
#     target_size=(150, 150),
#     batch_size=32
# )

In [8]:
#optimizer = SGD(lr=0.002,momentum=0.8)
#model.compile(optimizer = optimizer , loss = "categorical_crossentropy", metrics=["accuracy"])

In [18]:
history_callback=model.fit( train_ds, steps_per_epoch=100, epochs=10, validation_data=validation_ds, validation_steps=50 )


Epoch 1/10
100/100 [==============================] - 22s 219ms/step - loss: 0.5510 - accuracy: 0.7305 - val_loss: 0.5872 - val_accuracy: 0.6890
Epoch 2/10
100/100 [==============================] - 21s 210ms/step - loss: 0.5303 - accuracy: 0.7435 - val_loss: 0.5649 - val_accuracy: 0.7110
Epoch 3/10
100/100 [==============================] - 19s 193ms/step - loss: 0.5400 - accuracy: 0.7315 - val_loss: 0.5711 - val_accuracy: 0.7150
Epoch 4/10
100/100 [==============================] - 21s 206ms/step - loss: 0.5295 - accuracy: 0.7450 - val_loss: 0.5517 - val_accuracy: 0.7140
Epoch 5/10
100/100 [==============================] - 21s 213ms/step - loss: 0.5184 - accuracy: 0.7465 - val_loss: 0.5772 - val_accuracy: 0.6960
Epoch 6/10
100/100 [==============================] - 24s 240ms/step - loss: 0.5197 - accuracy: 0.7355 - val_loss: 0.5743 - val_accuracy: 0.7130
Epoch 7/10
100/100 [==============================] - 20s 204ms/step - loss: 0.4885 - accuracy: 0.7655 - val_loss: 0.5499 - val_ac

In [27]:
median_acc = np.median(history_callback.history["accuracy"])
sd_loss = np.std(history_callback.history["loss"])
mean_vloss = np.mean(history_callback.history["val_loss"])
avg_val_acc_5_10 = np.mean(history_callback.history["val_accuracy"][4:])
print(f"median training acc: {median_acc}")
print(f"std of training loss: {sd_loss}")
print(f"mean of validation loss: {mean_vloss}")
print(f"avg val acc for epochs 5 to 10: {avg_val_acc_5_10}")

median training acc: 0.7457500100135803
std of training loss: 0.024408338916025778
mean of validation loss: 0.5623502135276794
avg val acc for epochs 5 to 10: 0.7144999901453654


In [28]:
# Data agumentation part
# We are not supposed to "re-create the model - we want to continue training the model we already started training"
# Apparently this means:
# Yes you don't need to recompile it. But even if you compile again, it doesn't reset the model you trained previously


In [29]:
train_gen = ImageDataGenerator(rescale=1./255,
                            rotation_range=40,
                            width_shift_range=0.2,
                            height_shift_range=0.2,
                            shear_range=0.2,
                            zoom_range=0.2,
                            horizontal_flip=True,
                            fill_mode='nearest')
train_ds = train_gen.flow_from_directory(os.path.join(DATAPATH,"train"),
class_mode='binary', batch_size=20, target_size=(150, 150))

validation_ds = train_gen.flow_from_directory(os.path.join(DATAPATH,"validation"),
 class_mode='binary', batch_size=20, target_size=(150, 150))

Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.


In [30]:
#model.add_metric(
#            keras.backend.std(inputs), name="std_of_activation", aggregation="mean"
#        )

In [31]:
history_callback = model.fit( train_ds, steps_per_epoch=100, epochs=10, validation_data=validation_ds, validation_steps=50 )

Epoch 1/10
100/100 [==============================] - 30s 295ms/step - loss: 0.6143 - accuracy: 0.6645 - val_loss: 0.6308 - val_accuracy: 0.6390
Epoch 2/10
100/100 [==============================] - 31s 309ms/step - loss: 0.6244 - accuracy: 0.6600 - val_loss: 0.5981 - val_accuracy: 0.6920
Epoch 3/10
100/100 [==============================] - 30s 297ms/step - loss: 0.6209 - accuracy: 0.6510 - val_loss: 0.6107 - val_accuracy: 0.6600
Epoch 4/10
100/100 [==============================] - 27s 272ms/step - loss: 0.6153 - accuracy: 0.6570 - val_loss: 0.5939 - val_accuracy: 0.6910
Epoch 5/10
100/100 [==============================] - 28s 279ms/step - loss: 0.6082 - accuracy: 0.6650 - val_loss: 0.5985 - val_accuracy: 0.6740
Epoch 6/10
100/100 [==============================] - 27s 266ms/step - loss: 0.6104 - accuracy: 0.6685 - val_loss: 0.5954 - val_accuracy: 0.6660
Epoch 7/10
100/100 [==============================] - 28s 277ms/step - loss: 0.6078 - accuracy: 0.6650 - val_loss: 0.6171 - val_ac

In [39]:
median_acc = np.median(history_callback.history["accuracy"])
sd_loss = np.std(history_callback.history["loss"])
mean_vloss = np.mean(history_callback.history["val_loss"])
avg_val_acc_5_10 = np.mean(history_callback.history["val_accuracy"][4:])
print(f"median training acc: {median_acc}")
print(f"std of training loss: {sd_loss}")
print(f"mean of validation loss: {mean_vloss}")
print(f"avg val acc for epochs 5 to 10: {avg_val_acc_5_10}")
#print(history_callback.history["val_accuracy"])

#print(history_callback.history["val_accuracy"][4:])


median training acc: 0.6650000214576721
std of training loss: 0.008172949186204842
mean of validation loss: 0.6026202142238617
avg val acc for epochs 5 to 10: 0.6768333315849304


In [18]:
#scores = model.evaluate(validation_ds)
#print(scores[1])

250/250 [==============================] - 35s 139ms/step - loss: 0.6436 - accuracy: 0.6284
0.6284000277519226


## 8.13 Summary

* We can use pre-trained models for general image classification
* Convolutional layers let us turn an image into a vector
* Dense layers use the vector to make the predictions
* Instead of training a model from scratch, we can use transfer learning and re-use already trained convolutional layers
* First, train a small model (150x150) before training a big one (299x299)
* Learning rate - how fast the model trians. Fast learners aren't always best ones
* We can save the best model using callbacks and checkpointing
* To avoid overfitting, use dropout and augmentation

## 8.14 Explore more

* Add more data, e.g. Zalando, etc (ADD LINKS)
* Albumentations - another way of generating augmentations
* Use PyTorch or MXNet instead of TensorFlow/Keras
* In addition to Xception, there are others architectures - try them 

Other projects:

* cats vs dogs
* Hotdog vs not hotdog
* Category of images

